# Исследование надежности заемщиков

Заказчик – кредитный отдел банка. Необходимо разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. В качестве исходных данных будем использовать статистику о платежеспособности клиентов.

Результаты исследования могут быть использованы при построении модели *кредитного скоринга* – системы, оценивающей способность потенциального заемщика вернуть кредит банку.

## 1. Изучение общей информации о данных

Для начала импортируем библиотеку **pandas**. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`. Путь к файлу:

`/datasets/data.csv`

In [1]:
import pandas as pd

data = pd.read_csv('datasets/data.csv')

Выведем первые 20 строчек датафрейма `data` на экран.

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме с помощью метода **`info()`**.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## 2. Предобработка данных

### 2.1 Удаление пропусков

Выведем количество пропущенных значений для каждого столбца. Для этого будем использовать комбинацию двух методов **`isna()`** и **`sum()`**.

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них – `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями – `total_income` – хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен мединанным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### 2.2 Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) – значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода **`abs()`**.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения.

Выведем перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Еще раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### 2.3 Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями для каждого типа занятости `income_type`.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### 2.4 Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода **`astype()`**.

In [13]:
data['total_income'] = data['total_income'].astype(int)

### 2.5 Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их все к нижнему регистру.

In [14]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных и удалим их.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### 2.6 Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:
- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

Напишем собственную функцию `categorize_income()` и применим ее к датафрейму при помощи метода **`apply()`**.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Далее создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:
- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Используем собственную функцию `categorize_purpose()` и метод **`apply()`**.

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## 3 Исследовательский анализ данных

### Шаг 3. Исследуйте данные и ответьте на вопросы

Для удобности и наглядности выведем обновленный dataset.

In [22]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


Как мы можем заметить, в исходном dataset добавились новые столбцы (параметры), которые мы получили ранее после прохождения категоризации данных (`total_income_category`, `purpose_category`). Эти новые параметры также будем использовать при дальнейшем анализе необходимых зависимостей.

Для начала оценим количество людей, имеющих и неимеющих задолженность. 

In [23]:
data.groupby('debt')['debt'].count()

debt
0    19599
1     1732
Name: debt, dtype: int64

Для более понятной оценки рассмотрим процентное соотношение должников к общей выборке.

In [24]:
debtors = data.loc[data['debt'] == 1]
debtor_percentage = (len(debtors) / len(data)) * 100

print(f'Количество людей, имеющих задолженность в процентах: {debtor_percentage:0.2f} %')

Количество людей, имеющих задолженность в процентах: 8.12 %


Для лучшей читабельности и наглядности представим данные в виде удобных сводных таблиц, в которых будут отражены основные параметры, оказывающие влияние друг на друга. Для большей универсальности построения данных таблиц напишем функцию, позволяющую это реализовывать. В качестве входных параметров функции примем исходный датасет `data` и столбец/столбцы, по которым группируют данные `index_column`. На выходе же получим сформированную таблицу с общим количеством людей по группам, с количеством должников по группам и процентное соотношение должников в каждой группе.

In [25]:
def form_pivot(data, index_column):
    data_pivot = data.pivot_table(index=[index_column], values=['debt'], aggfunc={'sum', 'count'})
    data_pivot['percent_debtors'] = (data_pivot['debt']['sum'] / data_pivot['debt']['count']) * 100
    data_pivot = data_pivot.sort_values('percent_debtors', ascending=False)
    return data_pivot

### 3.1 Исследование зависимости между количеством детей и возврата кредита в срок

Рассмотрим зависимость между количеством детей и возвратом кредита в срок. Построим сводную таблицу с нужными столбцами для обобщения данных и более наглядного их представления. Воспользуемся ранее написанной функцией __`form_pivot()`__.

In [26]:
pivot_children = form_pivot(data, 'children')

pivot_children

debt       percent_debtors
          count   sum                
children                             
4            41     4        9.756098
2          2052   194        9.454191
1          4808   444        9.234609
3           330    27        8.181818
0         14091  1063        7.543822
5             9     0        0.000000

**Вывод:** 

Поскольку в исходном датасете имеются категории, полученные при группировке по количеству детей в семьях, с недостаточным количеством наблюдений, то невозможно сделать полноценный вывод. В данном случае будем рассматривать только семьи с 2-мя детьми, одним ребенком и семьи без детей, потому как количество данных для этих групп можно считать приблизительно одинаковыми. Производя оценку по этим категориям, можно сделать вывод, что при увеличении количества детей также возрастает и процент должников в группе. При большем количестве детей в семье веротяность возврата кредита в срок уменьшается.

### 3.2 Исследование зависимости между семейным положением и возвратом кредита в срок

Теперь рассмотрим зависимость семейного положения от возврата кредита в срок. Выведем интересующие данные также в форме таблицы.

In [27]:
pivot_family_status = form_pivot(data, 'family_status')

pivot_family_status

debt      percent_debtors
                       count  sum                
family_status                                    
Не женат / не замужем   2796  273        9.763948
гражданский брак        4134  385        9.313014
женат / замужем        12261  927        7.560558
в разводе               1189   84        7.064760
вдовец / вдова           951   63        6.624606

**Вывод:** 

Объединим рассмотренные группы в две большие категории: **заёмщики, которые находятся или были в браке**, и **заемщики, которые не были в браке**. При такой группировке можно заметить, что первой категории характерен более низкий процент должников.

### 3.3 Исследование зависимости между уровнем дохода и возвратом кредита в срок

Необходимо также учесть такой важный показатель, как уровень дохода, который может являться ключевым при оценке возврата кредита в срок. Рассмотрим данные в виде сводной таблицы.

Для удобства анализа зависимости рассмотрим несколько категорий, сформированных ранее в отдельном столбце `total_income_category`:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [28]:
pivot_income_cat = form_pivot(data, 'total_income_category')

pivot_income_cat

debt       percent_debtors
                       count   sum                
total_income_category                             
E                         22     2        9.090909
C                      15921  1353        8.498210
A                         25     2        8.000000
B                       5014   354        7.060231
D                        349    21        6.017192

**Вывод:** 

Поскольку в исходном датасете имеются категории, полученные при группировке по уровню дохода, с недостаточным количеством наблюдений, то невозможно сделать полноценный вывод. Выберем две категории (**"B"** и **"C"**), которые соответствуют уровню заработка выше среднего и среднему. Исходя их этого, категория заемщиков с уровнем дохода выше среднего более склонна к закрытию кредита в срок.

### 3.4 Исследование вляиния целей кредита на закрытие его в срок

Немаловажным фактором также является цель кредита, которая может обуславливать его размер. Воспользуемся сформированным столбцом `purpose_category` для построения сводной таблицы.

In [29]:
pivot_purpose_cat = form_pivot(data, 'purpose_category')

pivot_purpose_cat

debt      percent_debtors
                          count  sum                
purpose_category                                    
операции с автомобилем     4279  400        9.347978
получение образования      3988  369        9.252758
проведение свадьбы         2313  183        7.911803
операции с недвижимостью  10751  780        7.255139

**Вывод:**

1. Группа людей, цель кредита которых явлются операции с автомобилем и получение образование, имеет наибольшее процентное содержание должников (__`9,34 %`__ и __`9,25 %`__ соответственно).
2. Самой многочисленной и наиболее надежной категорией в плане возврата кредита в срок является группа людей, проводящая операции с недвижимостью (__`7,3 %`__).

Таким образом, цель кредита может являться одним из определяющих факторов при его вероятности возврата в срок.

## Общий вывод

В данной работе были проанализированы данные клиентов банка, произведена их предобработка с удалением дубликатов и заполнением пропусков в данных. Необходимо было выявить основные зависимости, влияющие на возврат кредита в срок. Среди них были рассмотрены:
- количества детей;
- семейное положение;
- уровень дохода;
- цели кредита.

Было выявлено, что
1. При увеличении количества детей увеличивается процент должников по выборке.
2. Семейное положение также оказывает влияние: для заёмщиков, которые находятся или были в браке, характерна более низкая вероятность невыплаты кредита в срок.
3. Категория людей с уровнем дохода выше среднего более склонна к закрытию кредита в срок.
4. Цель кредита также зависит от вероятности наличия долга: для "операций с недвижимостью" и "проведения свадьбы" был замечен более низкий процент должников.

При оценке возврата кредита в срок наиболее платежеспособной является группа людей:
- не имеющая детей или имеющая 5 детей(что может являться большой редкостью);
- в текущей момент женат/замужем, в разводе или вдовец/вдова;
- имеющая доход ниже среднего или выше среднего;
- целью которых явлется проведение свадьбы или операций с недвижмостью.

Для более точных результатов анализа рекомендуется увеличить объем данных по многодетным семьям и для категорий заемщиков с уровнем дохода ниже среднего.